# 🗣️ Model E — Patient-Facing Guidance & Next-Steps

**Purpose:** Translate system decisions into calm, clear, non-diagnostic guidance for the patient. Model E never reasons medically — it only communicates.

| | |
|---|---|
| **Input** | Structured symptoms (Model B) + priority level (Model D) + key flags |
| **Output** | A patient-facing message — plain, calm, voice-ready |
| **Model** | Google Gemini AI (controlled generation, no fine-tuning) |
| **Audience** | The patient — directly |

### Pipeline Position
```
Model B + Model D Output              Model E Output
(Symptoms + Priority)
         │
         ▼
┌─────────────────────────┐     ┌──────────────────────────────────────┐
│  priority: HIGH         │     │ "Your symptoms need prompt attention. │
│  symptoms: chest pain   │────►│  Please proceed to Emergency, Desk 3. │
│  red_flags: True        │     │  A healthcare professional will see   │
│  context answers        │     │  you shortly..."                      │
└─────────────────────────┘     └──────────────────────────────────────┘
                                              │
                                              ▼
                                     Spoken to patient
                                     Displayed on screen
```

> ⚠️ **Hard Rules:** Never names a disease. Never suggests a cause. Never gives medical advice. Never overrides the system's priority decision.

---
## 📦 Section 1 — Install & Imports

In [ ]:
!pip install -q -U google-generativeai

import json
import re
import datetime
from dataclasses import dataclass, field, asdict
from typing import List, Optional

import google.generativeai as genai
from google.colab import userdata

print("✅ Dependencies ready.")

---
## 🔑 Section 2 — API Key

In [ ]:
# Add your key via: left sidebar → 🔑 Secrets → "GEMINI_API_KEY"
try:
    genai.configure(api_key=userdata.get("GEMINI_API_KEY"))
    print("✅ API key loaded from Colab Secrets.")
except Exception:
    genai.configure(api_key="YOUR_API_KEY_HERE")  # fallback only
    print("⚠️  API key set manually. Use Colab Secrets for security.")

---
## 🤖 Section 3 — Model Initialisation

In [ ]:
GEMINI_MODEL_NAME = "gemini-1.5-flash"

gemini_model = genai.GenerativeModel(
    model_name        = GEMINI_MODEL_NAME,
    generation_config = genai.types.GenerationConfig(
        temperature       = 0.3,   # Slight warmth for natural patient-friendly phrasing
        max_output_tokens = 180,   # Keep messages short — 3 to 5 sentences max
    ),
    safety_settings = [
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_ONLY_HIGH"},
        {"category": "HARM_CATEGORY_HARASSMENT",        "threshold": "BLOCK_ONLY_HIGH"},
        {"category": "HARM_CATEGORY_HATE_SPEECH",       "threshold": "BLOCK_ONLY_HIGH"},
    ]
)

print(f"✅ Gemini initialised: {GEMINI_MODEL_NAME} | temperature=0.3")

---
## 🗂️ Section 4 — Data Structures

Two simple dataclasses: one for input (compiled from Models B and D), one for output.

In [ ]:
@dataclass
class GuidanceInput:
    """
    Everything Model E needs. Compiled from Model B and Model D.
    Model E does not re-infer anything — it only reads what is here.
    """
    # From Model D
    priority           : str             = "MEDIUM"   # HIGH | MEDIUM | LOW
    # From Model B
    chief_complaint    : str             = ""
    duration           : str             = ""
    severity           : str             = ""
    associated_symptoms: List[str]       = field(default_factory=list)
    red_flags_present  : Optional[bool]  = None
    # Optional context
    patient_language   : str             = "english"  # 'english' | 'kinyarwanda'
    location_context   : str             = ""         # e.g. "Emergency, Desk 3"
    uncertainty_flag   : bool            = False      # True if data was sparse/incomplete


@dataclass
class GuidanceOutput:
    """
    Model E's patient-facing message.
    """
    message            : str   # The message to speak or display to the patient
    priority_used      : str   # Priority level that drove the message
    language           : str   # Language the message was generated in


print("✅ Data structures defined.")

---
## 📋 Section 5 — Safe Fallback Templates

If the API call fails, or if data is too sparse, the system falls back to pre-approved template messages. Templates are audited, deterministic, and always safe.

In [ ]:
# ── Fallback templates — used when API fails or data is insufficient ──────────
# These are plain, safe, pre-approved messages for each priority level.
FALLBACK_TEMPLATES = {
    "HIGH": (
        "Your symptoms need prompt medical attention. "
        "Please proceed to the nearest staff member or emergency desk immediately. "
        "If you feel worse at any time, notify a staff member right away."
    ),
    "MEDIUM": (
        "Thank you for sharing your symptoms. "
        "A healthcare professional will see you shortly. "
        "Please take a seat and wait to be called. "
        "If your condition changes or worsens, please let a staff member know."
    ),
    "LOW": (
        "Thank you. Your information has been recorded. "
        "Please wait in the waiting area and you will be called when it is your turn. "
        "If you have any concerns while waiting, feel free to speak to a staff member."
    ),
    "UNKNOWN": (
        "Thank you for your information. "
        "Please wait while we complete the next step. "
        "A staff member will assist you shortly."
    ),
}

# ── Kinyarwanda fallback templates ───────────────────────────────────────────
FALLBACK_TEMPLATES_KIN = {
    "HIGH": (
        "Ibimenyetso byawe bisaba ubuvuzi bwihutirwa. "
        "Mwihangane, muzajya guturikirwa na muganga vuba vuba. "
        "Niba imiterere yawe ihinduka cyangwa irushaho gukomera, menyesha umukozi vuba."
    ),
    "MEDIUM": (
        "Murakoze gushiraho ibimenyetso byanyu. "
        "Muganga azababona vuba. "
        "Mwicare mukomeze gutegereza. "
        "Niba hari impinduka, menyesha umukozi."
    ),
    "LOW": (
        "Murakoze. Amakuru yanyu yanditswe. "
        "Mwicare mukomeze gutegereza kugeza igihe muzahamagalwa. "
        "Niba mufite ibibazo, mubwire umukozi."
    ),
    "UNKNOWN": (
        "Murakoze gutanga amakuru. "
        "Mwihangane turangije intambwe ikurikira. "
        "Umukozi azabafasha vuba."
    ),
}

print("✅ Fallback templates loaded (English + Kinyarwanda).")

---
## 📝 Section 6 — Prompt

The system prompt enforces all hard constraints once. The user prompt injects the system decision at runtime — the model explains it, never overrides it.

In [ ]:
SYSTEM_PROMPT = """\
You are a healthcare system assistant in a hospital.
Your only job is to explain the next steps to a patient using the system's decision.

=== HARD RULES (NEVER VIOLATE) ===
- Do NOT name any disease, condition, or diagnosis.
- Do NOT suggest a cause for the patient's symptoms.
- Do NOT give any medical advice or treatment suggestions.
- Do NOT change, escalate, or downgrade the priority level given to you.
- Do NOT use medical terminology the patient would not understand.
- Keep the message calm, clear, and reassuring.
- Write 3 to 5 short sentences only.
- If any key information is missing, do not guess — defer with:
  "Please wait while we complete the next step."
- Output the patient message only — no labels, no explanations, no extra text.
"""


# ── Few-shot examples (template-LLM hybrid approach) ─────────────────────────
# These are injected as conversation history so Gemini learns the expected
# style and tone before seeing the real input.
FEW_SHOT_HISTORY = [
    {
        "role": "user",
        "parts": ["""\
Priority: HIGH
Chief complaint: chest pain
Duration: 2 hours
Severity: severe
Red flag: yes
Location: Emergency, Desk 3
Language: english"""
        ]
    },
    {
        "role": "model",
        "parts": ["""\
Your symptoms need prompt medical attention. \
Please proceed to Emergency, Desk 3 right away. \
A healthcare professional will see you shortly. \
If you feel worse at any time, please notify a staff member immediately."""
        ]
    },
    {
        "role": "user",
        "parts": ["""\
Priority: MEDIUM
Chief complaint: headache
Duration: 2 days
Severity: moderate
Red flag: no
Location: Waiting Area B
Language: english"""
        ]
    },
    {
        "role": "model",
        "parts": ["""\
Thank you for sharing your information. \
Please take a seat in Waiting Area B and a healthcare professional will see you soon. \
We have noted your symptoms and you will be called when it is your turn. \
If your condition changes or worsens while you wait, please let a staff member know."""
        ]
    },
    {
        "role": "user",
        "parts": ["""\
Priority: LOW
Chief complaint: sore throat
Duration: 3 days
Severity: mild
Red flag: no
Location: General Outpatient
Language: english"""
        ]
    },
    {
        "role": "model",
        "parts": ["""\
Thank you. Your information has been recorded. \
Please wait in the General Outpatient area and you will be called when it is your turn. \
If you have any concerns while waiting, please speak to a staff member."""
        ]
    },
]


def build_prompt(inp: GuidanceInput) -> str:
    """Build the user-turn prompt from a GuidanceInput."""
    lines = [
        f"Priority: {inp.priority}",
        f"Chief complaint: {inp.chief_complaint or 'not specified'}",
        f"Duration: {inp.duration or 'not specified'}",
        f"Severity: {inp.severity or 'not specified'}",
        f"Red flag: {'yes' if inp.red_flags_present else 'no' if inp.red_flags_present is False else 'unknown'}",
        f"Other symptoms: {', '.join(inp.associated_symptoms) if inp.associated_symptoms else 'none'}",
        f"Location: {inp.location_context or 'not specified'}",
        f"Language: {inp.patient_language}",
    ]
    if inp.uncertainty_flag:
        lines.append("Note: Some patient data was incomplete. Keep the message general.")
    return "\n".join(lines)


print("✅ Prompt and few-shot examples defined.")

---
## 🛡️ Section 7 — Output Safety Check

A lightweight post-generation scan that flags if the model has accidentally included forbidden content (diagnostic language, medical advice, etc.).

In [ ]:
# Words and phrases that must never appear in a patient-facing message
FORBIDDEN_PHRASES = [
    # Diagnostic language
    "you have", "you may have", "this is", "this could be", "this looks like",
    "likely", "probably", "possibly", "diagnosis", "condition is",
    # Disease names (a sample — extend as needed)
    "heart attack", "stroke", "cancer", "infection", "diabetes",
    "hypertension", "pneumonia", "appendicitis",
    # Medical advice
    "take", "medication", "painkiller", "rest", "drink water",
    "avoid", "do not eat", "apply",
    # Urgency inflation
    "life-threatening", "critical", "emergency", "dying",
]


def safety_check(message: str) -> tuple:
    """
    Scan the generated message for forbidden content.

    Returns:
        (is_safe: bool, violations: list of matched phrases)
    """
    lowered    = message.lower()
    violations = [p for p in FORBIDDEN_PHRASES if p in lowered]
    return len(violations) == 0, violations


print("✅ Safety check defined.")
print(f"   Monitoring {len(FORBIDDEN_PHRASES)} forbidden phrases.")

---
## 🚀 Section 8 — Core Guidance Generation Function

In [ ]:
def generate_patient_guidance(
    guidance_input : GuidanceInput,
    verbose        : bool = True,
) -> dict:
    """
    Generate a patient-facing next-steps message.

    Args:
        guidance_input : Compiled input from Models B and D.
        verbose        : Print the message.

    Returns:
        dict with keys: output (GuidanceOutput), message, safe, violations, success, error
    """
    priority = guidance_input.priority.upper()

    if verbose:
        print("\n" + "═" * 58)
        print(" 🗣️   PATIENT GUIDANCE — MODEL E")
        print("═" * 58)
        print(f"  Priority   : {priority}")
        print(f"  Complaint  : {guidance_input.chief_complaint or '(not specified)'}")
        print(f"  Language   : {guidance_input.patient_language}")
        print(f"  Red flag   : {guidance_input.red_flags_present}")
        print(f"  Uncertain  : {guidance_input.uncertainty_flag}")

    # ── Hard fallback: uncertainty flag or unknown priority ───────────────────
    if guidance_input.uncertainty_flag or priority not in ("HIGH", "MEDIUM", "LOW"):
        templates = FALLBACK_TEMPLATES_KIN if guidance_input.patient_language == "kinyarwanda" \
                    else FALLBACK_TEMPLATES
        message = templates.get("UNKNOWN")
        if verbose:
            print("\n  ⚠️  Uncertainty flag set — using safe fallback message.")
            _print_message(message)
        output = GuidanceOutput(message=message, priority_used=priority,
                                language=guidance_input.patient_language)
        return {"output": output, "message": message, "safe": True,
                "violations": [], "success": True, "error": None}

    # ── Call Gemini ───────────────────────────────────────────────────────────
    try:
        if verbose:
            print("\n  🔄 Calling Gemini API...")

        # Prime the chat with system prompt + few-shot examples
        history = [{"role": "user", "parts": [SYSTEM_PROMPT]},
                   {"role": "model", "parts": ["Understood. I will write only the patient message — calm, clear, and with no diagnosis or medical advice."]}]
        history += FEW_SHOT_HISTORY

        chat     = gemini_model.start_chat(history=history)
        response = chat.send_message(build_prompt(guidance_input))
        message  = response.text.strip()

    except Exception as e:
        # Fall back to template on API error
        templates = FALLBACK_TEMPLATES_KIN if guidance_input.patient_language == "kinyarwanda" \
                    else FALLBACK_TEMPLATES
        message   = templates.get(priority, templates["UNKNOWN"])
        error_msg = str(e)
        if verbose:
            print(f"  ❌ API error: {error_msg} — using fallback template.")
        output = GuidanceOutput(message=message, priority_used=priority,
                                language=guidance_input.patient_language)
        return {"output": output, "message": message, "safe": True,
                "violations": [], "success": False, "error": error_msg}

    # ── Safety check ──────────────────────────────────────────────────────────
    is_safe, violations = safety_check(message)

    if not is_safe:
        # Replace with a safe fallback — never expose a flagged message to the patient
        templates = FALLBACK_TEMPLATES_KIN if guidance_input.patient_language == "kinyarwanda" \
                    else FALLBACK_TEMPLATES
        message   = templates.get(priority, templates["UNKNOWN"])
        if verbose:
            print(f"  ⚠️  Safety check FAILED. Violations: {violations}")
            print("      Replaced with pre-approved fallback message.")

    output = GuidanceOutput(message=message, priority_used=priority,
                            language=guidance_input.patient_language)

    if verbose:
        _print_message(message, is_safe)

    return {"output": output, "message": message, "safe": is_safe,
            "violations": violations, "success": True, "error": None}


def _print_message(message: str, is_safe: bool = True):
    safety_label = "✅ SAFE" if is_safe else "🔄 REPLACED (fallback)"
    print(f"\n  ─── PATIENT MESSAGE [{safety_label}] " + "─" * 18)
    print(f"\n  '{message}'\n")


print("✅ generate_patient_guidance() defined.")

---
## 🔗 Section 9 — Full Pipeline Integration (Models B + D → E)

In [ ]:
def run_from_pipeline(
    model_b_output   : dict,
    model_d_output   : dict,
    patient_language : str  = "english",
    location_context : str  = "",
    verbose          : bool = True,
) -> dict:
    """
    Compile outputs from Models B and D and generate patient guidance.

    Args:
        model_b_output   : Dict from Model B's extract_clinical_information().
        model_d_output   : Dict from Model D's score_patient_risk().
        patient_language : 'english' or 'kinyarwanda'.
        location_context : Where the patient should go (e.g. 'Emergency, Desk 3').
        verbose          : Print progress.

    Returns:
        Model E output dict.
    """
    ext   = model_b_output.get("extraction_dict", {})
    score = model_d_output.get("score_dict", {})

    guidance_input = GuidanceInput(
        priority            = score.get("priority", "MEDIUM"),
        chief_complaint     = ext.get("chief_complaint", ""),
        duration            = ext.get("duration", ""),
        severity            = ext.get("severity", ""),
        associated_symptoms = ext.get("associated_symptoms", []),
        red_flags_present   = ext.get("red_flags_present"),
        patient_language    = patient_language,
        location_context    = location_context,
        uncertainty_flag    = score.get("confidence", 1.0) < 0.4,
    )

    if verbose:
        print("🔗 Models B + D → Model E")

    return generate_patient_guidance(guidance_input, verbose=verbose)


# ─────────────────────────────────────────────────────────────────────────────
# USAGE:
#   model_b_result = extract_clinical_information(transcript)
#   model_d_result = run_from_model_b(model_b_result)
#   model_e_result = run_from_pipeline(model_b_result, model_d_result,
#                                      patient_language="english",
#                                      location_context="Emergency, Desk 3")
# ─────────────────────────────────────────────────────────────────────────────

print("✅ run_from_pipeline() defined.")

---
## 🧪 Section 10 — Test Cases

In [ ]:
# ── Test 1: HIGH — severe chest pain, red flag, English ───────────────────────
result_1 = generate_patient_guidance(GuidanceInput(
    priority            = "HIGH",
    chief_complaint     = "chest pain",
    duration            = "2 hours",
    severity            = "severe",
    associated_symptoms = ["shortness of breath"],
    red_flags_present   = True,
    patient_language    = "english",
    location_context    = "Emergency, Desk 3",
))

In [ ]:
# ── Test 2: MEDIUM — moderate headache, English ───────────────────────────────
result_2 = generate_patient_guidance(GuidanceInput(
    priority            = "MEDIUM",
    chief_complaint     = "headache",
    duration            = "2 days",
    severity            = "moderate",
    associated_symptoms = ["nausea"],
    red_flags_present   = False,
    patient_language    = "english",
    location_context    = "Waiting Area B",
))

In [ ]:
# ── Test 3: LOW — mild sore throat, English ───────────────────────────────────
result_3 = generate_patient_guidance(GuidanceInput(
    priority         = "LOW",
    chief_complaint  = "sore throat",
    duration         = "3 days",
    severity         = "mild",
    red_flags_present= False,
    patient_language = "english",
    location_context = "General Outpatient",
))

In [ ]:
# ── Test 4: MEDIUM — abdominal pain, Kinyarwanda ──────────────────────────────
result_4 = generate_patient_guidance(GuidanceInput(
    priority            = "MEDIUM",
    chief_complaint     = "ububabare mu nda",
    duration            = "kuva ejo",
    severity            = "uburemere",
    associated_symptoms = ["gushaka kuruka"],
    red_flags_present   = False,
    patient_language    = "kinyarwanda",
    location_context    = "Waiting Area A",
))

In [ ]:
# ── Test 5: Uncertain — sparse data, uncertainty flag triggered ───────────────
result_5 = generate_patient_guidance(GuidanceInput(
    priority         = "MEDIUM",
    chief_complaint  = "leg hurts",
    patient_language = "english",
    uncertainty_flag = True,   # sparse data → safe fallback used
))

---
## 📤 Section 11 — Export

In [ ]:
from google.colab import files

def export_guidance(results: list, filepath: str = "model_e_guidance.json"):
    """Save guidance results to JSON and download."""
    export = {
        "timestamp" : datetime.datetime.now().isoformat(),
        "model"     : GEMINI_MODEL_NAME,
        "total"     : len(results),
        "results"   : [
            {
                "index"     : i + 1,
                "priority"  : r["output"].priority_used,
                "language"  : r["output"].language,
                "message"   : r["message"],
                "safe"      : r["safe"],
                "violations": r["violations"],
                "success"   : r["success"],
            }
            for i, r in enumerate(results)
        ]
    }
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(export, f, ensure_ascii=False, indent=2)
    print(f"\n✅ Saved to: {filepath}")
    files.download(filepath)


# Export all test results:
# export_guidance([result_1, result_2, result_3, result_4, result_5])

print("✅ Export function ready. Uncomment the last line to download results.")

---
## 📝 Notes

| Topic | Detail |
|---|---|
| **Few-shot examples** | Three primed examples (HIGH, MEDIUM, LOW) are injected as conversation history before the real input. This is the template-LLM hybrid approach — Gemini learns style from examples, not from fine-tuning. |
| **Safety check** | Every generated message is scanned for forbidden phrases before it reaches the patient. If flagged, the message is silently replaced with a pre-approved fallback. |
| **Uncertainty flag** | If Model D's confidence is below 0.4, the uncertainty flag is set and a neutral holding message is used instead of generating a specific one. |
| **Fallback templates** | Available in both English and Kinyarwanda. They are static, audited, and always safe — used on API errors, safety failures, or uncertain inputs. |
| **Temperature = 0.3** | Slightly warmer than other models — allows natural, human-sounding phrasing while still staying within guardrails. |
| **No re-inference** | Model E reads `guidance_input` and explains it. It does not re-evaluate symptoms, reassess priority, or reason about the patient's condition. |
| **Location context** | Pass the physical location (e.g. `"Emergency, Desk 3"`) so the message tells the patient exactly where to go. |